In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import math

In [2]:
dfMaximals = pd.read_csv("./playlistData/maximal_T4_20.csv")
dfPlaylists = pd.read_csv("./playlistData/spotify_dataset.csv", on_bad_lines='skip')
dfMaximals.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,0,"('Pendulum', '9,000 Miles')","('Crystal Castles', 'Celestica')","('Justice', 'D.A.N.C.E - MSTRKRFT Remix')","('Justice', 'Genesis')",0.966667,0.958333,0.966667,0.966667,0.966667,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
1,1,"('Pendulum', '9,000 Miles')","('Crystal Castles', 'Celestica')","('Justice', 'D.A.N.C.E - MSTRKRFT Remix')","('The Prodigy', 'Invaders Must Die')",0.966667,0.958333,0.966667,0.966667,0.966667,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
2,2,"('Pendulum', '9,000 Miles')","('Crystal Castles', 'Celestica')","('Justice', 'D.A.N.C.E - MSTRKRFT Remix')","('Crystal Castles', 'Magic Spells')",0.966667,0.958333,0.966667,0.966667,0.966667,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
3,3,"('Pendulum', '9,000 Miles')","('Crystal Castles', 'Celestica')","('Justice', 'D.A.N.C.E - MSTRKRFT Remix')","('Pendulum', 'Mutiny')",0.966667,0.958333,0.966667,0.966667,0.966667,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
4,4,"('Pendulum', '9,000 Miles')","('Crystal Castles', 'Celestica')","('Justice', 'D.A.N.C.E - MSTRKRFT Remix')","('Ratatat', 'Nostrand')",0.966667,0.958333,0.966667,0.966667,0.966667,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667


In [3]:
def parseTuple(songAndArtist):
    artist, song = "", ""
    foundOpen, foundComma, foundSpace, foundClose = False, False, False, False
    for i in songAndArtist:
        if (not foundOpen and i == "("):
            foundOpen = True
            continue
            
        if (not foundComma and i == ","):
            foundComma = True
            continue
            
        if (not foundSpace and i == " "):
            foundSpace = True
            continue
            
        if (not foundClose and i == ")"):
            foundClose = True
        
        if (foundOpen and not foundComma and i != "'"):
            artist += i
            continue
            
        if (foundComma and foundSpace and not foundClose and i != "'"):
            song += i
            continue
            
        
            
        if (foundClose and artist[0] == " "):
            artist = artist[1:-1]
            
        if (foundClose and song[0] == " "):
            song = song[1:-1]
        
    return (artist, song)

In [4]:
quadIntrest = {}
quadRules = {}
tripleIntrest = {}
tripleRules = {}
pairIntrest = {}
pairRules = {}
maximalsLength = dfMaximals.shape[0]
count = 0
for i in range(maximalsLength):
    if (not dfMaximals.isnull().iloc[i][3]):
        if (not dfMaximals.isnull().iloc[i][4]):
            # quads
            quad = [parseTuple(dfMaximals.iloc[i][1]), parseTuple(dfMaximals.iloc[i][2]), parseTuple(dfMaximals.iloc[i][3]), parseTuple(dfMaximals.iloc[i][4])]
            for j in range(4):
                subsection = quad.copy()
                subsection.remove(quad[j])
                key = (tuple(subsection), quad[j])
                quadRules[tuple(subsection)] = quad[j]
                quadIntrest[key] = dfMaximals.iloc[i][j + 5]
                
            for k in range(4):
                if (k == 0):
                    sub1 = [quad[0], quad[1], quad[2]]
                elif (k == 1):
                    sub1 = [quad[0], quad[1], quad[3]]
                elif (k == 2):
                    sub1 = [quad[0], quad[2], quad[3]]
                elif (k == 3):
                    sub1 = [quad[1], quad[2], quad[3]]
                    
                for j in sub1:
                    subsection = sub1.copy()
                    subsection.remove(j)
                    key = (tuple(subsection), j)
                    if (key in list(tripleIntrest.keys())):
                        currentIntrest = tripleIntrest[key]
                        if (currentIntrest < dfMaximals.iloc[i][k + 9]):
                            tripleIntrest[key] = dfMaximals.iloc[i][k + 9]
                    else:
                        tripleRules[tuple(subsection)] = j
                        tripleIntrest[key] = dfMaximals.iloc[i][k + 9]
                    
                
            for k in range(6):
                if (k == 0):
                    sub2 = [quad[0], quad[1]]
                elif (k == 1):
                    sub2 = [quad[0], quad[2]]
                elif (k == 2):
                    sub2 = [quad[0], quad[3]]
                elif (k == 3):
                    sub2 = [quad[1], quad[2]]
                elif (k == 4):
                    sub2 = [quad[1], quad[3]]
                elif (k == 5):
                    sub2 = [quad[2], quad[3]]
                    
                for j in sub2:
                    subsection = sub2.copy()
                    subsection.remove(j)
                    key = (subsection[0], j)
                    if (key in list(pairIntrest.keys())):
                        currentIntrest = pairIntrest[key]
                        if (currentIntrest < dfMaximals.iloc[i][k + 25]):
                            pairIntrest[key] = dfMaximals.iloc[i][k + 25]
                    else:
                        pairRules[subsection[0]] = j
                        pairIntrest[key] = dfMaximals.iloc[i][k + 25]
            
        #triples
        triple = [parseTuple(dfMaximals.iloc[i][1]), parseTuple(dfMaximals.iloc[i][2]), parseTuple(dfMaximals.iloc[i][3])]
        for j in range(3):
            subsection = triple.copy()
            subsection.remove(triple[j])
            key = (tuple(subsection), triple[j])
            if (key in list(tripleIntrest.keys())):
                currentIntrest = tripleIntrest[key]
                if (currentIntrest < dfMaximals.iloc[i][j + 5]):
                    tripleIntrest[key] = dfMaximals.iloc[i][j + 5]
            else:
                tripleRules[tuple(subsection)] = triple[j]
                tripleIntrest[key] = dfMaximals.iloc[i][j + 5]
                
        for k in range(3):
            if (k == 0):
                sub2 = [triple[0], triple[1]]
            elif (k == 1):
                sub2 = [triple[0], triple[2]]
            elif (k == 2):
                sub2 = [triple[1], triple[2]]
                
            for j in sub2:
                count += 1
                subsection = sub2.copy()
                subsection.remove(j)
                key = (subsection[0], j)
                if (key in list(pairIntrest.keys())):
                    currentIntrest = pairIntrest[key]
                    if (currentIntrest < dfMaximals.iloc[i][k + 8]):
                        pairIntrest[key] = dfMaximals.iloc[i][k + 8]
                else:
                    pairRules[subsection[0]] = j
                    pairIntrest[key] = dfMaximals.iloc[i][k + 8]
        
    #pairs
    pair = [parseTuple(dfMaximals.iloc[i][1]), parseTuple(dfMaximals.iloc[i][2])]
    for j in range(2):
        count += 1
        subsection = pair.copy()
        subsection.remove(pair[j])
        key = (subsection[0], pair[j])
        if (key in list(pairIntrest.keys())):
            currentIntrest = pairIntrest[key]
            if (currentIntrest < dfMaximals.iloc[i][j + 5]):
                pairIntrest[key] = dfMaximals.iloc[i][j + 5]
        else:
            pairRules[subsection[0]] = pair[j]
            pairIntrest[key] = dfMaximals.iloc[i][j + 5]
        
        
print(count)

24696


In [5]:
quadKeys = list(quadIntrest.keys())
tripleKeys = list(tripleIntrest.keys())
pairKeys = list(pairIntrest.keys())
print(len(quadKeys))
print(len(tripleKeys))
print(len(pairKeys))
print(pairKeys[0][0])

12212
3618
894
('CrystalCastles', 'Celestic')


In [6]:
def GenerateRecomendations(numRequested, playlist, calculateQuads):
    count = 0
    recommendations = []
    pairSingeltons = []
    triplePairs = []
    quadTriples = []
    for i in pairKeys:
        pairSingeltons.append(i[0])
    for i in tripleKeys:
        triplePairs.append(i[0])
    for i in quadKeys:
        quadTriples.append(i[0])
    
    # generate all single for pair rules
    for i in range(len(playlist)):
        single = playlist[i]
        if (single in pairSingeltons):
            recExists = False
            for j in recommendations:
                if (pairRules[single] == j[0]):
                    recExists = True
                    if (pairIntrest[(single, pairRules[single])] > j[1]):
                        recommendations.append((pairRules[single], pairIntrest[(single, pairRules[single])]))
            if (not recExists):
                recommendations.append((pairRules[single], pairIntrest[(single, pairRules[single])]))
                
    if (len(recommendations) == 0):
        return []
                
    # generate all pairs for triple rules
    for i in range(len(playlist)):
        for j in range(i, len(playlist)):
            pair = (playlist[i], playlist[j])
            for l in triplePairs:
                if (pair[0] in l[0] and pair[1] in l[0]):
                    pair = l[0]
                    break
                    
            if (pair in triplePairs):
                recExists = False
                for k in recommendations:
                    if (tripleRules[pair] == k[0]):
                        recExists = True
                        if (tripleIntrest[(pair, tripleRules[pair])] > k[1]):
                            recommendations.append((tripleRules[pair], tripleIntrest[(pair, tripleRules[pair])]))
                if (not recExists):
                    recommendations.append((tripleRules[pair], tripleIntrest[(pair, tripleRules[pair])]))
                
    # generate all triples for quad rules
    if (calculateQuads):
        for i in range(len(playlist)):
            for j in range(i, len(playlist)):
                for k in range(j, len(playlist)):
                    triple = (playlist[i], playlist[j], playlist[k])
                    for l in quadTriples:
                        if (triple[0] in l[0] and triple[1] in l[0] and triple[2] in l[0]):
                            triple = l[0]
                            break

                    if (triple in quadTriples):
                        recExists = False
                        for l in recommendations:
                            if (triple[0] == l[0]):
                                recExists = True
                                if (quadIntrest[(triple, quadRules[triple])] > l[1]):
                                    recommendations.append((quadRules[triple], quadIntrest[(triple, quadRules[triple])]))
                        if (not recExists):
                            recommendations.append((quadRules[triple], quadIntrest[(triple, quadRules[triple])]))
                    
    return recommendations

In [7]:
playlists = []
playlistNames = []
dfPlaylists.iloc[0][3]
for i in range(500): # change to the whole thing at end
    currentPLName = dfPlaylists.iloc[i][3]
    if (currentPLName not in playlistNames):
        playlistNames.append(currentPLName)
        playlists.append([])
        
    playlists[playlistNames.index(currentPLName)].append((dfPlaylists.iloc[i][1], dfPlaylists.iloc[i][2]))
    
print(len(playlists))

8


In [8]:
rec = GenerateRecomendations(2, playlists[7], False)

In [9]:
print(rec)

[(('CrystalCastles', 'Vanishe'), 0.9666666666666668)]


In [14]:
pairSingeltons = []
for i in pairKeys:
    pairSingeltons.append(i[0])

both = []
for i in playlists[7]:
    if (i in pairSingeltons):
        both.append(i)
        
print(i)

[('CrystalCastles', 'Celestic'), ('Pendulum', '9,000 Miles'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('Pendulum', '9,000 Miles'), ('Justice', 'Genesis'), ('Pendulum', '9,000 Miles'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('CrystalCastles', 'Celestic'), ('Justice', 'Genesis'), ('CrystalCastles', 'Celestic'), ('Justice', 'Genesis'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('TheProdigy', 'Invaders Must Di'), ('Pendulum', '9,000 Miles'), ('TheProdigy', 'Invaders Must Di'), ('CrystalCastles', 'Celestic'), ('TheProdigy', 'Invaders Must Di'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('CrystalCastles', 'Magic Spell'), ('Pendulum', '9,000 Miles'), ('CrystalCastles', 'Magic Spell'), ('CrystalCastles', 'Celestic'), ('CrystalCastles', 'Magic Spell'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('Pendulum', 'Mutiny'), ('Pendulum', '9,000 Miles'), ('Pendulum', 'Mutiny'), ('CrystalCastles', 'Celestic'), ('Pendulum', 'Mutiny'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('Ratatat', 'Nostr